# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [3]:
load_dotenv()
BASE_URL = "https://openai.vocareum.com/v1"
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
CHROMA_OPENAI_API_KEY=os.getenv("CHROMA_OPENAI_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")

### VectorDB Instance

In [4]:
chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [5]:
# If picking something different than openai,
# make sure you use the same when loading it
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_base=BASE_URL, api_key=CHROMA_OPENAI_API_KEY)

In [18]:
chroma_client.delete_collection("udaplay")

In [19]:
collection = chroma_client.create_collection(
   name="udaplay",
   embedding_function=embedding_fn
)

### Add documents

In [20]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) -genre: {game['Genre']} -publisher: {game['Publisher']} - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [21]:
collection.count()

15

In [22]:
collection.query(
    query_texts=["racing"],
    n_results=2 # how many results to return
)

{'ids': [['003', '001']],
 'embeddings': None,
 'documents': [['[PlayStation 3] Gran Turismo 5 (2010) -genre: Racing -publisher: Sony Computer Entertainment - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
   '[PlayStation 1] Gran Turismo (1997) -genre: Racing -publisher: Sony Computer Entertainment - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'Publisher': 'Sony Computer Entertainment',
    'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
    'Genre': 'Racing',
    'YearOfRelease': 2010,
    'Platform': 'PlayStation 3',
    'Name': 'Gran Turismo 5'},
   {'Platform': 'PlayStation 1',
    'YearOfRelease': 1997,
    'Description': 'A realistic racing simulator featur